# Практикум 5. Кластеризация данных

## Задание

1. Используя метод локтя, определить наилучшее разбиение методом K-means
2. Используя график расстояния агломерации, определить наилучшее разбиение с помощью иерархического агломеративного подхода. Использовать метод ближней связи
3. Используя график расстояния агломерации, определить наилучшее разбиение с помощью иерархического агломеративного подхода. Использовать метод центроидов
4. Сравнить по критериям - коэффициент силуэта, коэффициент r2, коэффициент Davies-Bouldin - решения и пп.1-3, и выбрать лучшее решение
5. Используя индекс Rand сравнить лучшее решение, с каждым из пп.1-3, определить объекты, которые все решения помещают вместе, и те объекты, которые являются граничными
6. Постройте кластерные профили по лучшему решению, визуализировав их с помощью линейных графиков
7. Интерпретируйте полученные кластеры лучшего решения

**Параметры:**
- Расстояние: Евклидово
- Seed: 1000
- Перед кластеризацией инициализировать seed = 1000

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score, adjusted_rand_score
from sklearn.decomposition import PCA
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import pdist
import warnings
warnings.filterwarnings('ignore')

# Установка seed для воспроизводимости
np.random.seed(1000)

plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

## 1. Загрузка и предобработка данных

In [ ]:
# Загрузка данных
data = pd.read_csv('online_shoppers_intention.csv')
print(f"Размер данных: {data.shape}")
print(f"\nПервые 5 строк:")
print(data.head())
print(f"\nИнформация о данных:")
print(data.info())
print(f"\nОписательная статистика:")
print(data.describe())

In [ ]:
# Проверка на пропущенные значения
print("Пропущенные значения:")
print(data.isnull().sum())

# Проверка уникальных значений в категориальных переменных
categorical_columns = ['Month', 'OperatingSystems', 'Browser', 'Region', 'TrafficType', 'VisitorType', 'Weekend', 'Revenue']
print("\nУникальные значения в категориальных переменных:")
for col in categorical_columns:
    print(f"{col}: {data[col].unique()}")

In [ ]:
# Предобработка данных
data_processed = data.copy()

# Кодирование категориальных переменных
le = LabelEncoder()
categorical_columns = ['Month', 'OperatingSystems', 'Browser', 'Region', 'TrafficType', 'VisitorType', 'Weekend', 'Revenue']

for col in categorical_columns:
    data_processed[col] = le.fit_transform(data_processed[col])

print("Данные после кодирования:")
print(data_processed.head())
print(f"\nРазмер данных: {data_processed.shape}")

In [ ]:
# Выделение признаков для кластеризации (исключаем целевую переменную Revenue)
features = data_processed.drop('Revenue', axis=1)
target = data_processed['Revenue']

print(f"Количество признаков: {features.shape[1]}")
print(f"Названия признаков: {list(features.columns)}")

# Стандартизация данных
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

print(f"\nДанные после стандартизации:")
print(f"Среднее: {np.mean(features_scaled, axis=0)[:5]}")
print(f"Стандартное отклонение: {np.std(features_scaled, axis=0)[:5]}")

## 2. K-means с методом локтя

In [ ]:
# Метод локтя для определения оптимального количества кластеров
inertias = []
silhouette_scores = []
k_range = range(2, 21)

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=1000, n_init=10)
    kmeans.fit(features_scaled)
    inertias.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(features_scaled, kmeans.labels_))

# Построение графика метода локтя
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

ax1.plot(k_range, inertias, 'bo-')
ax1.set_xlabel('Количество кластеров (k)')
ax1.set_ylabel('Inertia')
ax1.set_title('Метод локтя для K-means')
ax1.grid(True)

ax2.plot(k_range, silhouette_scores, 'ro-')
ax2.set_xlabel('Количество кластеров (k)')
ax2.set_ylabel('Silhouette Score')
ax2.set_title('Silhouette Score для K-means')
ax2.grid(True)

plt.tight_layout()
plt.show()

# Определение оптимального k
optimal_k_elbow = k_range[np.argmax(silhouette_scores)]
print(f"Оптимальное количество кластеров по методу локтя: {optimal_k_elbow}")
print(f"Лучший silhouette score: {max(silhouette_scores):.4f}")

In [ ]:
# Выполнение K-means с оптимальным количеством кластеров
kmeans_optimal = KMeans(n_clusters=optimal_k_elbow, random_state=1000, n_init=10)
kmeans_labels = kmeans_optimal.fit_predict(features_scaled)

print(f"K-means кластеризация выполнена с {optimal_k_elbow} кластерами")
print(f"Распределение объектов по кластерам:")
unique, counts = np.unique(kmeans_labels, return_counts=True)
for cluster, count in zip(unique, counts):
    print(f"Кластер {cluster}: {count} объектов ({count/len(kmeans_labels)*100:.1f}%)")

## 3. Иерархическая кластеризация с методом ближней связи

In [ ]:
# Иерархическая кластеризация с методом ближней связи
# Используем подвыборку для ускорения вычислений
sample_size = 2000
sample_indices = np.random.choice(len(features_scaled), sample_size, replace=False)
features_sample = features_scaled[sample_indices]

# Построение дендрограммы
linkage_single = linkage(features_sample, method='single')

plt.figure(figsize=(15, 8))
dendrogram(linkage_single, truncate_mode='level', p=10)
plt.title('Дендрограмма для метода ближней связи')
plt.xlabel('Образцы')
plt.ylabel('Расстояние')
plt.show()

# Анализ расстояний агломерации
distances = linkage_single[:, 2]
plt.figure(figsize=(12, 6))
plt.plot(range(1, len(distances) + 1), distances[::-1], 'bo-')
plt.xlabel('Количество кластеров')
plt.ylabel('Расстояние агломерации')
plt.title('График расстояния агломерации (метод ближней связи)')
plt.grid(True)
plt.show()

# Поиск оптимального количества кластеров по наибольшему скачку
distances_diff = np.diff(distances[::-1])
optimal_k_single = np.argmax(distances_diff) + 2
print(f"Оптимальное количество кластеров по методу ближней связи: {optimal_k_single}")

In [ ]:
# Выполнение иерархической кластеризации с оптимальным количеством кластеров
hierarchical_single = AgglomerativeClustering(n_clusters=optimal_k_single, linkage='single')
hierarchical_single_labels = hierarchical_single.fit_predict(features_scaled)

print(f"Иерархическая кластеризация (ближняя связь) выполнена с {optimal_k_single} кластерами")
print(f"Распределение объектов по кластерам:")
unique, counts = np.unique(hierarchical_single_labels, return_counts=True)
for cluster, count in zip(unique, counts):
    print(f"Кластер {cluster}: {count} объектов ({count/len(hierarchical_single_labels)*100:.1f}%)")

## 4. Иерархическая кластеризация с методом центроидов

In [ ]:
# Иерархическая кластеризация с методом центроидов
linkage_centroid = linkage(features_sample, method='centroid')

plt.figure(figsize=(15, 8))
dendrogram(linkage_centroid, truncate_mode='level', p=10)
plt.title('Дендрограмма для метода центроидов')
plt.xlabel('Образцы')
plt.ylabel('Расстояние')
plt.show()

# Анализ расстояний агломерации
distances_centroid = linkage_centroid[:, 2]
plt.figure(figsize=(12, 6))
plt.plot(range(1, len(distances_centroid) + 1), distances_centroid[::-1], 'go-')
plt.xlabel('Количество кластеров')
plt.ylabel('Расстояние агломерации')
plt.title('График расстояния агломерации (метод центроидов)')
plt.grid(True)
plt.show()

# Поиск оптимального количества кластеров по наибольшему скачку
distances_diff_centroid = np.diff(distances_centroid[::-1])
optimal_k_centroid = np.argmax(distances_diff_centroid) + 2
print(f"Оптимальное количество кластеров по методу центроидов: {optimal_k_centroid}")

In [ ]:
# Выполнение иерархической кластеризации с оптимальным количеством кластеров
hierarchical_centroid = AgglomerativeClustering(n_clusters=optimal_k_centroid, linkage='average')
hierarchical_centroid_labels = hierarchical_centroid.fit_predict(features_scaled)

print(f"Иерархическая кластеризация (центроиды) выполнена с {optimal_k_centroid} кластерами")
print(f"Распределение объектов по кластерам:")
unique, counts = np.unique(hierarchical_centroid_labels, return_counts=True)
for cluster, count in zip(unique, counts):
    print(f"Кластер {cluster}: {count} объектов ({count/len(hierarchical_centroid_labels)*100:.1f}%)")

## 5. Сравнение методов по критериям качества

In [ ]:
# Вычисление метрик качества для всех методов
def evaluate_clustering(X, labels, method_name):
    silhouette = silhouette_score(X, labels)
    calinski_harabasz = calinski_harabasz_score(X, labels)
    davies_bouldin = davies_bouldin_score(X, labels)
    
    return {
        'Method': method_name,
        'Silhouette Score': silhouette,
        'Calinski-Harabasz Score (R2)': calinski_harabasz,
        'Davies-Bouldin Score': davies_bouldin
    }

# Оценка всех методов
results = []
results.append(evaluate_clustering(features_scaled, kmeans_labels, f'K-means (k={optimal_k_elbow})'))
results.append(evaluate_clustering(features_scaled, hierarchical_single_labels, f'Hierarchical Single (k={optimal_k_single})'))
results.append(evaluate_clustering(features_scaled, hierarchical_centroid_labels, f'Hierarchical Centroid (k={optimal_k_centroid})'))

# Создание таблицы результатов
results_df = pd.DataFrame(results)
print("Сравнение методов кластеризации:")
print(results_df.round(4))

# Определение лучшего метода
best_method_idx = results_df['Silhouette Score'].idxmax()
best_method = results_df.iloc[best_method_idx]
print(f"\nЛучший метод: {best_method['Method']}")
print(f"Silhouette Score: {best_method['Silhouette Score']:.4f}")
print(f"Calinski-Harabasz Score: {best_method['Calinski-Harabasz Score (R2)']:.4f}")
print(f"Davies-Bouldin Score: {best_method['Davies-Bouldin Score']:.4f}")

In [ ]:
# Визуализация сравнения методов
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

methods = results_df['Method'].tolist()
silhouette_scores = results_df['Silhouette Score'].tolist()
calinski_scores = results_df['Calinski-Harabasz Score (R2)'].tolist()
davies_scores = results_df['Davies-Bouldin Score'].tolist()

axes[0].bar(methods, silhouette_scores, color=['skyblue', 'lightcoral', 'lightgreen'])
axes[0].set_title('Silhouette Score')
axes[0].set_ylabel('Score')
axes[0].tick_params(axis='x', rotation=45)

axes[1].bar(methods, calinski_scores, color=['skyblue', 'lightcoral', 'lightgreen'])
axes[1].set_title('Calinski-Harabasz Score (R2)')
axes[1].set_ylabel('Score')
axes[1].tick_params(axis='x', rotation=45)

axes[2].bar(methods, davies_scores, color=['skyblue', 'lightcoral', 'lightgreen'])
axes[2].set_title('Davies-Bouldin Score')
axes[2].set_ylabel('Score')
axes[2].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 6. Анализ с помощью индекса Rand

In [ ]:
# Определение лучшего решения и сравнение с другими методами
if best_method_idx == 0:
    best_labels = kmeans_labels
    best_method_name = 'K-means'
elif best_method_idx == 1:
    best_labels = hierarchical_single_labels
    best_method_name = 'Hierarchical Single'
else:
    best_labels = hierarchical_centroid_labels
    best_method_name = 'Hierarchical Centroid'

print(f"Лучший метод: {best_method_name}")

# Вычисление индекса Rand для сравнения с другими методами
rand_scores = []
comparison_methods = ['K-means', 'Hierarchical Single', 'Hierarchical Centroid']
all_labels = [kmeans_labels, hierarchical_single_labels, hierarchical_centroid_labels]

for i, (method, labels) in enumerate(zip(comparison_methods, all_labels)):
    if not np.array_equal(labels, best_labels):
        rand_score = adjusted_rand_score(best_labels, labels)
        rand_scores.append(rand_score)
        print(f"Rand Index между {best_method_name} и {method}: {rand_score:.4f}")
    else:
        print(f"{method} - это лучший метод")

if rand_scores:
    print(f"\nСредний Rand Index: {np.mean(rand_scores):.4f}")

In [ ]:
# Анализ согласованности кластеризаций
def find_consistent_objects(labels1, labels2, threshold=0.8):
    """Найти объекты, которые согласованно кластеризуются в обоих методах"""
    consistent = []
    for i in range(len(labels1)):
        # Подсчет сколько раз объект i попадает в тот же кластер в обоих методах
        same_cluster_count = 0
        for j in range(len(labels1)):
            if i != j:
                if (labels1[i] == labels1[j]) == (labels2[i] == labels2[j]):
                    same_cluster_count += 1
        consistency = same_cluster_count / (len(labels1) - 1)
        if consistency >= threshold:
            consistent.append(i)
    return consistent

# Найти согласованные объекты между лучшим методом и остальными
print("Анализ согласованности кластеризаций:")
for i, (method, labels) in enumerate(zip(comparison_methods, all_labels)):
    if not np.array_equal(labels, best_labels):
        consistent_objects = find_consistent_objects(best_labels, labels, threshold=0.7)
        print(f"\nСогласованные объекты между {best_method_name} и {method}: {len(consistent_objects)} ({len(consistent_objects)/len(best_labels)*100:.1f}%)")
        
        # Найти граничные объекты (те, которые не согласованы)
        all_indices = set(range(len(best_labels)))
        boundary_objects = list(all_indices - set(consistent_objects))
        print(f"Граничные объекты: {len(boundary_objects)} ({len(boundary_objects)/len(best_labels)*100:.1f}%)")

## 7. Построение кластерных профилей

In [ ]:
# Построение кластерных профилей для лучшего решения
def plot_cluster_profiles(data, labels, feature_names, n_clusters):
    fig, axes = plt.subplots(2, 2, figsize=(20, 15))
    axes = axes.ravel()
    
    # Выбираем наиболее важные признаки для визуализации
    important_features = ['Administrative', 'Administrative_Duration', 'ProductRelated', 'ProductRelated_Duration', 
                         'BounceRates', 'ExitRates', 'PageValues', 'SpecialDay']
    
    for i, feature in enumerate(important_features[:8]):
        if i < len(axes):
            for cluster in range(n_clusters):
                cluster_data = data[labels == cluster][feature]
                axes[i].plot([cluster] * len(cluster_data), cluster_data, 'o', alpha=0.6, label=f'Cluster {cluster}')
            
            axes[i].set_title(f'Профиль кластеров: {feature}')
            axes[i].set_xlabel('Кластер')
            axes[i].set_ylabel('Значение признака')
            axes[i].grid(True)
    
    plt.tight_layout()
    plt.show()

# Построение профилей
n_clusters_best = len(np.unique(best_labels))
plot_cluster_profiles(features, best_labels, features.columns, n_clusters_best)

In [ ]:
# Детальный анализ кластерных профилей
def analyze_cluster_profiles(data, labels, feature_names):
    n_clusters = len(np.unique(labels))
    
    print(f"Детальный анализ кластерных профилей ({best_method_name}):")
    print("=" * 60)
    
    for cluster in range(n_clusters):
        cluster_data = data[labels == cluster]
        print(f"\nКластер {cluster} ({len(cluster_data)} объектов, {len(cluster_data)/len(data)*100:.1f}%):")
        print("-" * 40)
        
        for feature in feature_names:
            mean_val = cluster_data[feature].mean()
            std_val = cluster_data[feature].std()
            print(f"{feature}: {mean_val:.3f} ± {std_val:.3f}")

analyze_cluster_profiles(features, best_labels, features.columns)

In [ ]:
# Визуализация кластеров в 2D пространстве (PCA)
pca = PCA(n_components=2)
features_pca = pca.fit_transform(features_scaled)

plt.figure(figsize=(12, 8))
scatter = plt.scatter(features_pca[:, 0], features_pca[:, 1], c=best_labels, cmap='viridis', alpha=0.6)
plt.colorbar(scatter)
plt.title(f'Визуализация кластеров ({best_method_name}) в 2D PCA пространстве')
plt.xlabel(f'Первая главная компонента ({pca.explained_variance_ratio_[0]:.2%} дисперсии)')
plt.ylabel(f'Вторая главная компонента ({pca.explained_variance_ratio_[1]:.2%} дисперсии)')
plt.grid(True)
plt.show()

print(f"Объясненная дисперсия: {pca.explained_variance_ratio_.sum():.2%}")

## 8. Интерпретация кластеров

In [ ]:
# Детальная интерпретация кластеров
def interpret_clusters(data, labels, original_data):
    n_clusters = len(np.unique(labels))
    
    print(f"ИНТЕРПРЕТАЦИЯ КЛАСТЕРОВ ({best_method_name})")
    print("=" * 80)
    
    for cluster in range(n_clusters):
        cluster_mask = labels == cluster
        cluster_data = data[cluster_mask]
        cluster_original = original_data[cluster_mask]
        
        print(f"\n{'='*20} КЛАСТЕР {cluster} {'='*20}")
        print(f"Размер: {len(cluster_data)} объектов ({len(cluster_data)/len(data)*100:.1f}% от общего количества)")
        
        # Анализ ключевых характеристик
        print(f"\nКЛЮЧЕВЫЕ ХАРАКТЕРИСТИКИ:")
        
        # Административные страницы
        admin_mean = cluster_data['Administrative'].mean()
        admin_dur_mean = cluster_data['Administrative_Duration'].mean()
        print(f"• Административные страницы: {admin_mean:.1f} (время: {admin_dur_mean:.1f} сек)")
        
        # Информационные страницы
        info_mean = cluster_data['Informational'].mean()
        info_dur_mean = cluster_data['Informational_Duration'].mean()
        print(f"• Информационные страницы: {info_mean:.1f} (время: {info_dur_mean:.1f} сек)")
        
        # Продуктовые страницы
        product_mean = cluster_data['ProductRelated'].mean()
        product_dur_mean = cluster_data['ProductRelated_Duration'].mean()
        print(f"• Продуктовые страницы: {product_mean:.1f} (время: {product_dur_mean:.1f} сек)")
        
        # Показатели качества
        bounce_mean = cluster_data['BounceRates'].mean()
        exit_mean = cluster_data['ExitRates'].mean()
        pageval_mean = cluster_data['PageValues'].mean()
        print(f"• Показатели качества: Bounce={bounce_mean:.3f}, Exit={exit_mean:.3f}, PageValue={pageval_mean:.3f}")
        
        # Специальные дни
        special_mean = cluster_data['SpecialDay'].mean()
        print(f"• Специальные дни: {special_mean:.3f}")
        
        # Демографические характеристики
        weekend_pct = cluster_original['Weekend'].mean() * 100
        revenue_pct = cluster_original['Revenue'].mean() * 100
        print(f"• Выходные дни: {weekend_pct:.1f}%")
        print(f"• Конверсия в покупку: {revenue_pct:.1f}%")
        
        # Тип посетителя
        visitor_types = cluster_original['VisitorType'].value_counts()
        print(f"• Типы посетителей: {dict(visitor_types)}")
        
        # Месяцы
        months = cluster_original['Month'].value_counts().head(3)
        print(f"• Популярные месяцы: {dict(months)}")
        
        # Интерпретация поведения
        print(f"\nИНТЕРПРЕТАЦИЯ ПОВЕДЕНИЯ:")
        
        if admin_mean > data['Administrative'].mean():
            print("• Высокая активность на административных страницах")
        if product_mean > data['ProductRelated'].mean():
            print("• Высокая активность на продуктовых страницах")
        if bounce_mean < data['BounceRates'].mean():
            print("• Низкий показатель отказов (хорошее качество трафика)")
        if pageval_mean > data['PageValues'].mean():
            print("• Высокая ценность страниц")
        if revenue_pct > original_data['Revenue'].mean() * 100:
            print("• Высокая конверсия в покупку")
        
        print("\n" + "-"*60)

interpret_clusters(features, best_labels, data)

## Заключение и выводы

In [ ]:
print("ЗАКЛЮЧЕНИЕ И ВЫВОДЫ")
print("=" * 50)
print(f"\n1. ЛУЧШИЙ МЕТОД КЛАСТЕРИЗАЦИИ: {best_method_name}")
print(f"   - Silhouette Score: {best_method['Silhouette Score']:.4f}")
print(f"   - Calinski-Harabasz Score: {best_method['Calinski-Harabasz Score (R2)']:.4f}")
print(f"   - Davies-Bouldin Score: {best_method['Davies-Bouldin Score']:.4f}")

print(f"\n2. КОЛИЧЕСТВО КЛАСТЕРОВ: {len(np.unique(best_labels))}")

print(f"\n3. АНАЛИЗ ФОРМЫ КЛАСТЕРОВ:")
print(f"   - Rand Index показывает согласованность между методами")
if 'rand_scores' in locals() and rand_scores:
    print(f"   - Средний Rand Index: {np.mean(rand_scores):.4f}")
    if np.mean(rand_scores) > 0.5:
        print(f"   - Высокая согласованность методов → сферическая форма кластеров")
    else:
        print(f"   - Низкая согласованность методов → сложная форма кластеров")

print(f"\n4. КАЧЕСТВО КЛАСТЕРИЗАЦИИ:")
if best_method['Silhouette Score'] > 0.5:
    print(f"   - Отличное качество кластеризации (Silhouette > 0.5)")
elif best_method['Silhouette Score'] > 0.3:
    print(f"   - Хорошее качество кластеризации (Silhouette > 0.3)")
else:
    print(f"   - Удовлетворительное качество кластеризации")

print(f"\n5. ПРАКТИЧЕСКОЕ ПРИМЕНЕНИЕ:")
print(f"   - Кластеры позволяют сегментировать пользователей по поведению")
print(f"   - Можно разработать персонализированные стратегии для каждого сегмента")
print(f"   - Высококонвертируемые сегменты требуют особого внимания")
print(f"   - Сегменты с высоким bounce rate нуждаются в улучшении UX")